# TruLens + Milvus

Setup:
To get up and running, you'll first need to install Docker and Milvus. Find instructions below:
* Docker Compose ([Instructions](https://docs.docker.com/compose/install/))
* Milvus Standalone ([Instructions](https://milvus.io/docs/install_standalone-docker.md))


Taken from : https://github.com/truera/trulens/blob/62896156df1a057f24527aabef04dfce3cd69631/trulens_eval/examples/expositional/vector-dbs/milvus/milvus_evals_build_better_rags.ipynb#L8

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [1]:
 pip install trulens-eval==0.12.0 llama_index==0.8.4 pymilvus==2.3.0 nltk==3.8.1 html2text==2020.1.16 tenacity==8.2.3 wikipedia sentence_transformers

Looking in indexes: https://gitlab.gitguardian.ovh/api/v4/projects/435/packages/pypi/simple
  Using cached wikipedia-1.4.0-py3-none-any.whl
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 2.3 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 kB 547.4 kB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.8/673.8 kB 1.0 MB/s eta 0:00:00 MB/s eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 987.8 kB/s eta 0:00:0036m

### Add API keys
For this quickstart, you will need Open AI and Huggingface keys

In [3]:
import os
os.environ["OPENAI_API_KEY"] = ""

### Import from LlamaIndex and TruLens

In [4]:
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores import MilvusVectorStore
from llama_index.llms import OpenAI
from llama_index import (
    VectorStoreIndex,
    SimpleWebPageReader,
    LLMPredictor,
    ServiceContext
)

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings

from tenacity import retry, stop_after_attempt, wait_exponential

from trulens_eval import TruLlama, Feedback, Tru, feedback
from trulens_eval.feedback import Groundedness
tru = Tru()


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### First we need to load documents. We can use SimpleWebPageReader

In [6]:
from llama_index import WikipediaReader

cities = [
    "Los Angeles", "Houston", "Honolulu", "Tucson", "Mexico City", 
    "Cincinatti", "Chicago"
]

wiki_docs = []
for city in cities:
    try:
        doc = WikipediaReader().load_data(pages=[city])
        wiki_docs.extend(doc)
    except Exception as e:
        print(f"Error loading page for city {city}: {e}")

### Now write down our test prompts

In [7]:
test_prompts = [
    "What's the best national park near Honolulu",
    "What are some famous universities in Tucson?",
    "What bodies of water are near Chicago?",
    "What is the name of Chicago's central business district?",
    "What are the two most famous universities in Los Angeles?",
    "What are some famous festivals in Mexico City?",
    "What are some famous festivals in Los Angeles?",
    "What professional sports teams are located in Los Angeles",
    "How do you classify Houston's climate?",
    "What landmarks should I know about in Cincinatti"
]


### Build a prototype RAG

In [9]:
vector_store = MilvusVectorStore(index_params={
        "index_type": "IVF_FLAT",
        "metric_type": "L2"
        },
        search_params={"nprobe": 20},
        overwrite=True)
llm = OpenAI(model="gpt-3.5-turbo")
embed_v12 = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
storage_context = StorageContext.from_defaults(vector_store = vector_store)
service_context = ServiceContext.from_defaults(embed_model = embed_v12, llm = llm)
index = VectorStoreIndex.from_documents(wiki_docs,
            service_context=service_context,
            storage_context=storage_context)
query_engine = index.as_query_engine(top_k = 5)

@retry(stop=stop_after_attempt(10), wait=wait_exponential(multiplier=1, min=4, max=10))
def call_query_engine(prompt):
        return query_engine.query(prompt)
for prompt in test_prompts:
    call_query_engine(prompt)

pytorch_model.bin:   0%|          | 0.00/471M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/16cc9e54df6e083272378abec2d75dc34d7a48b5276db3ccc050d18de672ac59?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1702881921&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjg4MTkyMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zZW50ZW5jZS10cmFuc2Zvcm1lcnMvcGFyYXBocmFzZS1tdWx0aWxpbmd1YWwtTWluaUxNLUwxMi12Mi8xNmNjOWU1NGRmNmUwODMyNzIzNzhhYmVjMmQ3NWRjMzRkN2E0OGI1Mjc2ZGIzY2NjMDUwZDE4ZGU2NzJhYzU5P3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=G-%7EBlmwbVTZkkTgz3tCDnuU8kTKHPUQw%7E1UtYMYuXI-Zl87LCNpF-%7Eu1EkeYVyCJa5gqNnS-ntibaEppXYMaDW7NrxpS28VmvJImAnYOA5qQnmZYsqN%7EBRsq%7Ecl3JW%7E-CD8itCYDcNQ2kgs8XpLsoRE-ZGhurIiWEAzhXlz9fbobDscO8S3DmOtTfgTQW6gI

KeyboardInterrupt: 

### Set up Evaluation.

In [ ]:
import numpy as np

# Initialize OpenAI-based feedback function collection class:
openai_gpt35 = feedback.OpenAI(model_engine="gpt-3.5-turbo")

# Define groundedness
grounded = Groundedness(groundedness_provider=openai_gpt35)
f_groundedness = Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness").on(
    TruLlama.select_source_nodes().node.text.collect() # context
).on_output().aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai_gpt35.relevance_with_cot_reasons, name = "Answer Relevance").on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(openai_gpt35.qs_relevance_with_cot_reasons, name = "Context Relevance").on_input().on(
    TruLlama.select_source_nodes().node.text
).aggregate(np.max)

### Find the best configuration.

In [ ]:
index_params = ["IVF_FLAT","HNSW"]
embed_v12 = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embed_ft3_v12 = HuggingFaceEmbeddings(model_name = "Sprylab/paraphrase-multilingual-MiniLM-L12-v2-fine-tuned-3")
embed_ada = OpenAIEmbeddings(model_name = "text-embedding-ada-002")
embed_models = [embed_v12, embed_ada]
top_ks = [1,3]
chunk_sizes = [200,500]

In [ ]:
import itertools
for index_param, embed_model, top_k, chunk_size in itertools.product(
    index_params, embed_models, top_ks, chunk_sizes
    ):
    if embed_model == embed_v12:
        embed_model_name = "v12"
    elif embed_model == embed_ft3_v12:
        embed_model_name = "ft3_v12"
    elif embed_model == embed_ada:
        embed_model_name = "ada"
    vector_store = MilvusVectorStore(index_params={
        "index_type": index_param,
        "metric_type": "L2"
        },
        search_params={"nprobe": 20},
        overwrite=True)
    llm = OpenAI(model="gpt-3.5-turbo")
    storage_context = StorageContext.from_defaults(vector_store = vector_store)
    service_context = ServiceContext.from_defaults(embed_model = embed_model, llm = llm, chunk_size=chunk_size)
    index = VectorStoreIndex.from_documents(wiki_docs,
            service_context=service_context,
            storage_context=storage_context)
    query_engine = index.as_query_engine(similarity_top_k = top_k)
    tru_query_engine = TruLlama(query_engine,
                    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance],
                    metadata={
                        'index_param':index_param,
                        'embed_model':embed_model_name,
                        'top_k':top_k,
                        'chunk_size':chunk_size
                        })
    @retry(stop=stop_after_attempt(10), wait=wait_exponential(multiplier=1, min=4, max=10))
    def call_tru_query_engine(prompt):
        return tru_query_engine.query(prompt)
    for prompt in test_prompts:
        call_tru_query_engine(prompt)

### Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

### Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all